## Conclusion
If the tfidf value of the top most salient terms in an LDA model aren't found or are below a threshold in a new Document, retrain the LDA model with the new document.

Dataset Used: [Bishop Dataset](https://www.kaggle.com/datasets/daishinkan002/bishop-topic-modelling-dataset)

In [ ]:
%pip install contractions pyLDAvis "pandas<2.0.0"

In [2]:
import pandas as pd
import numpy as np

In [3]:
bishop = pd.read_csv("Bishop.csv", index_col=0)
bishop["text"][0]

' problem searching patterns data fundamental one long successful history instance extensive astronomical observations tycho brahe century allowed johannes kepler discover empirical laws planetary motion turn provided springboard development classical mechanics similarly discovery regularities atomic spectra played key role development veriﬁcation quantum physics early twentieth century ﬁeld pattern recognition concerned automatic discovariance ery regularities data use computer algorithms use regularities take actions classifying data different categories consider example recognizing handwritten digits illustrated figure digit corresponds pixel image represented vector comprising real numbers goal build machine take vector input produce identity digit output nontrivial problem due wide variability handwriting could introduction figure examples hand written dig taken zip codes tackled using handcrafted rules heuristics distinguishing digits based shapes strokes practice approach leads 

In [ ]:
%pip install nltk

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.add('subject')
stop_words.add('http')

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

import contractions

import re
import string

In [6]:
def preprocess(text):
    text = re.sub('[%s]' % re.escape(string.punctuation.replace('+', '')), '' , text)

    text = text.lower()
    text = nltk.word_tokenize(text)

    expanded_cont = []
    for i in text:
        expanded_cont.append(contractions.fix(i))
    text = expanded_cont

    text = [word for word in text if word not in stop_words]
    text = [lemmatizer.lemmatize(word) for word in text]
    return text

In [7]:
from pprint import pprint

pprint(preprocess(bishop["text"][0])[:10])

['problem',
 'searching',
 'pattern',
 'data',
 'fundamental',
 'one',
 'long',
 'successful',
 'history',
 'instance']


In [8]:
bishop['tokens'] = bishop['text'].apply(lambda x : preprocess(x))
bishop.head()

,topic,text,tokens
0,[introduction],problem searching patterns data fundamental o...,"[problem, searching, pattern, data, fundamenta..."
1,"[example, polynomial, curve, fitting]",begin introducing simple regression problem s...,"[begin, introducing, simple, regression, probl..."
2,"[probability, theory]",key concept ﬁeld pattern recognition uncertai...,"[key, concept, ﬁeld, pattern, recognition, unc..."
3,"[probability, theory, probability, densities]",well considering probabilities deﬁned discret...,"[well, considering, probability, deﬁned, discr..."
4,"[probability, theory, expectations, covariances]",one important operations involving probabilit...,"[one, important, operation, involving, probabi..."


In [9]:
data_words = list(bishop.tokens)

In [10]:
import gensim
import gensim.corpora as corpora

from gensim.models import TfidfModel
from gensim.models import Phrases, phrases

In [11]:
def make_phrases(tokens: list[str]):
    bigram_ph = Phrases(data_words, min_count=5, threshold=50)
    trigram_ph = Phrases(bigram_ph[data_words], threshold=50)

    bigram = phrases.Phraser(bigram_ph)
    trigram = phrases.Phraser(trigram_ph)

    data_bigrams = [bigram[doc] for doc in tokens]
    data_trigrams = [trigram[bigram[doc]] for doc in tokens]
    return data_trigrams

def train_tfidf(data: list[list[str]]):
    id2word = corpora.Dictionary(data)
    corpus = [id2word.doc2bow(word) for word in data]

    tfidfModel = TfidfModel(corpus, id2word = id2word)
    return tfidfModel

def tfidf_filter(data: list[list[str]], model: TfidfModel, id2word = None):
    if id2word == None:
        id2word = corpora.Dictionary(data)
    corpus = [id2word.doc2bow(word) for word in data]

    low_val = 0.03
    low_val_words = []
    for bow in corpus:
        low_val_words += [id for id, value in model[bow] if value < low_val]

    id2word.filter_tokens(bad_ids=low_val_words)

    return ([id2word.doc2bow(doc) for doc in data], id2word)

In [12]:
data_words = make_phrases(data_words)
tfidf_model = train_tfidf(data_words)
data_corpus, id2word = tfidf_filter(data_words, tfidf_model)
print(id2word)
print(data_corpus[0][:50])

Dictionary<2352 unique tokens: ['ability', 'affect', 'astronomical', 'atomic', 'board']...>
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 4), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 4), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)]


In [13]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=data_corpus,
                                            id2word=id2word,
                                            num_topics=10,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')

In [14]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [ ]:
pyLDAvis.enable_notebook()

vis = gensimvis.prepare(lda_model, data_corpus, id2word, mds='mmds', R=20)

In [16]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.017328  0.445871       1        1  17.585508
7      0.029920 -0.342002       2        1  14.732183
8     -0.302657 -0.172987       3        1  11.778180
5     -0.188208  0.184385       4        1   9.841592
6      0.151617 -0.155536       5        1   9.235873
2      0.325841 -0.053912       6        1   9.155397
1      0.172934  0.180591       7        1   8.530434
0     -0.072428 -0.133298       8        1   7.634342
3     -0.207491  0.028553       9        1   6.043203
9      0.073144  0.018334      10        1   5.463289, topic_info=                        Term       Freq      Total Category  logprob  loglift
623                  journal  58.000000  58.000000  Default  20.0000  20.0000
2326               mit_press  66.000000  66.000000  Default  19.0000  19.0000
1376        gaussian_process  69.000000  69.000000  Default  18.0000  18.0000
1136  support_vector_machine  41.000000  41.000000  Default  17.0000  17.0000
2182                   split  36.000000  36.000000  Default  16.0000  16.0000
...                      ...        ...        ...      ...      ...      ...
957                  recover   2.160608   2.773104  Topic10  -5.1919   2.6575
2275              occurrence   1.877569   2.489925  Topic10  -5.3323   2.6248
444                   radius   4.670903   8.600644  Topic10  -4.4209   2.2966
615                     draw   3.089971   5.355634  Topic10  -4.8341   2.3571
1759              reasonably   2.413572   4.791974  Topic10  -5.0812   2.2213

[269 rows x 6 columns], token_table=      Topic      Freq               Term
term                                    
2168      2  0.353969           absolute
2168      5  0.589948           absolute
2120      5  0.809044       acceleration
2012      2  0.991273           accepted
1         3  0.215774             affect
...     ...       ...                ...
1055      3  0.827157  worth_emphasizing
2274      9  0.834035        yellowstone
986       2  0.929313                ﬂat
827       3  0.978080         ﬂexibility
705      10  0.747004            ﬂipping

[313 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 8, 9, 6, 7, 3, 2, 1, 4, 10])

In [17]:
# Next step: Copy a msdn or msvc doc webpage and pass it through all the steps. And see the plot for it
with open("ms-text.md", "r") as f:
    ms_text = f.read()
    f.close()

ms_text = preprocess(ms_text)

print(ms_text)

['title', 'inside', 'native', 'application', 'msdate', '11012006', 'mstopic', 'conceptual', 'msprod', 'windowssysinternals', 'mstechnology', 'systemutilities', 'description', 'article', 'i am', 'going', 'describe', 'native', 'application', 'built', 'work', 'inside', 'native', 'application', 'mark', 'russinovich', 'published', 'november', '1', '2006', 'introduction', 'familiarity', 'nt', 'architecture', 'probably', 'aware', 'api', 'win32', 'application', 'use', 'is not', 'real', 'nt', 'api', 'nt', 'operating', 'environment', 'include', 'posix', 'os2', 'win32', 'talk', 'client', 'application', 'via', 'apis', 'talk', 'nt', 'using', 'nt', 'native', 'api', 'native', 'api', 'mostly', 'undocumented', '25', '250', 'function', 'described', 'window', 'nt', 'device', 'driver', 'kit', 'people', 'do not', 'know', 'however', 'native', 'application', 'exist', 'nt', 'client', 'operating', 'environment', 'program', 'speak', 'native', 'nt', 'api', 'cannot', 'use', 'operating', 'environment', 'apis', 'li

In [ ]:
ms_text = make_phrases([ms_text])
print(ms_text)
id2word.add_documents(ms_text)
ms_corpus = [id2word.doc2bow(text) for text in ms_text]
print(ms_corpus)
print(id2word)

lda_model = gensim.models.ldamodel.LdaModel(corpus=data_corpus+ms_corpus,
                                            id2word=id2word,
                                            num_topics=10,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')

ms_lda = lda_model[ms_corpus[0]]
print(ms_lda)

vis = gensimvis.prepare(lda_model, ms_corpus, id2word, mds='mmds', R=20)

In [19]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.110000  0.359105       1        1  93.172262
5      0.170202 -0.226622       2        1   3.914890
4      0.165727  0.204020       3        1   1.078683
7     -0.050666 -0.080220       4        1   0.637888
3     -0.286614  0.114232       5        1   0.611399
8     -0.009070  0.149546       6        1   0.484354
9      0.355781 -0.000398       7        1   0.035372
2     -0.251855 -0.091969       8        1   0.029662
0      0.126910  0.001979       9        1   0.024118
1     -0.110412 -0.429673      10        1   0.011372, topic_info=                          Term      Freq     Total Category  logprob  loglift
1341               homogeneous  0.000000  0.000000  Default  20.0000  20.0000
2191            base_classiﬁer  0.000000  0.000000  Default  19.0000  19.0000
846       correctly_normalized  0.000000  0.000000  Default  18.0000  18.0000
655              communication  0.000000  0.000000  Default  17.0000  17.0000
234              deterministic  1.000000  1.000000  Default  16.0000  16.0000
...                        ...       ...       ...      ...      ...      ...
2322  international_conference  0.000625  0.027776  Topic10  -4.4816   5.2882
2349                university  0.000625  0.027776  Topic10  -4.4816   5.2882
2098                   science  0.000759  0.034877  Topic10  -4.2884   5.2538
1506                    jordan  0.001041  0.052012  Topic10  -3.9716   5.1709
817                     sparse  0.000780  0.039919  Topic10  -4.2603   5.1468

[243 rows x 6 columns], token_table=      Topic      Freq                       Term
term                                            
2363      1  1.131502                        api
2366      1  1.011804                application
471       1  1.049952                     aspect
2372      1  0.945330                    autochk
2191      1  1.490543             base_classiﬁer
403       2  1.399723                       beam
2378      1  0.970920                      build
655       1  1.552058              communication
474       1  1.642475               conditioning
846       1  1.304410       correctly_normalized
2196      1  1.614858                   designed
2057      1  0.989710           detailed_balance
234       1  0.855602              deterministic
2404      1  0.845391                     device
2135      1  1.584036                     devise
1719      1  0.976841                     driver
2412      1  1.062817                environment
2422      1  1.131502                       file
2427      1  1.131502                   function
2235      1  1.139171       hierarchical_mixture
322       1  1.561242                  highlight
1341      1  1.457470                homogeneous
927       1  1.073009                initialized
2233      1  1.627955                   interact
553       1  1.953650  keep_notation_uncluttered
2160      2  1.830984       leapfrog_integration
2450      1  0.845391                       like
2475      1  0.970920                       must
2478      1  1.011804                     native
2482      1  1.048786                         nt
1446      1  1.039280                       open
1596      1  0.799890               parent_child
1597      1  0.696752                     passed
2508      1  0.945330                    program
275       1  1.517460                  published
2517      1  1.062817                        run
2518      1  1.131502                    runtime
1454      1  0.706800              unfortunately
2561      1  1.062817                      win32, R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 6, 5, 8, 4, 9, 10, 3, 1, 2])